# Exploration de la source de données

In [26]:
import pandas as pd
from datetime import date, timedelta
import os

In [27]:
# Racine des fichiers quotidiens
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'

# Dates de disponibilité des fichiers
START_DATE = date(2020, 1, 22)
END_DATE = date(2020, 3, 21)

# Répertoire de sauvegarde des fichiers bruts
RAWFILES_DIR = '../data/raw/'
PROCESSED_DIR = '../data/processed/'

# Fichier principal
ALL_DATA_FILE = 'all_data.csv'

#TODO: A remplacer par la lecture du fichier env.yaml

## Boucle de récupération des fichiers

In [28]:
delta = END_DATE - START_DATE       # as timedelta

for i in range(delta.days + 1):
    day = START_DATE + timedelta(days=i)
    day_label = day.strftime("%m-%d-%Y")
    #print(day_label)
    virus_df = pd.read_csv(BASE_URL.format(day_label), sep=',', parse_dates=['Last Update'])
    virus_df.to_csv(os.path.join(RAWFILES_DIR, day_label + '.csv'), index=False)

In [25]:
virus_df.dtypes

Province/State            object
Country/Region            object
Last Update       datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
Latitude                 float64
Longitude                float64
dtype: object

## Constitution de la table de références lat/long

In [29]:
import glob

df_list = []

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',')
    if 'Latitude' in virus_df.columns and 'Longitude' in virus_df.columns:
        df_list.append(virus_df)

all_df = pd.concat(df_list)

# Création d'une table de références pour les lat/long
(all_df[['Province/State', 'Country/Region', 'Latitude', 'Longitude']]
 .drop_duplicates(subset=['Province/State', 'Country/Region'])
 .sort_values(by=['Country/Region', 'Province/State'])
 .to_csv(os.path.join(PROCESSED_DIR, 'lat_long_table.csv'), index=False)
)

## Construction d'une table unique

In [30]:
data_catalog = {
    'Last Update': ['<M8[ns]'],
    'Confirmed': ['float64', 'int64'],
    'Deaths': ['float64', 'int64'],
    'Recovered': ['float64', 'int64'],
    'Latitude': ['float64'],
    'Longitude': ['float64'],
}

In [31]:
df_list = []
latlong_df = pd.read_csv(os.path.join(PROCESSED_DIR, 'lat_long_table.csv'))

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',', parse_dates=['Last Update'])
    if not('Latitude' in virus_df.columns and 'Longitude' in virus_df.columns):
        virus_df = virus_df.merge(latlong_df, on=['Province/State', 'Country/Region'], how='left')
        
    for field, types in data_catalog.items():
        assert virus_df[field].dtypes in types, f"Bad type for {field} in {file}"
        
    df_list.append(virus_df.assign(source=os.path.basename(file)))

all_df = pd.concat(df_list)

# Sauvegarde de la table totale
all_df.to_csv(os.path.join(PROCESSED_DIR, 'all_data.csv'), index=False)

In [32]:
all_df


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,source
0,Anhui,Mainland China,2020-01-22 17:00:00,1.0,NaN,NaN,31.8257,117.2264,01-22-2020.csv
1,Beijing,Mainland China,2020-01-22 17:00:00,14.0,NaN,NaN,40.1824,116.4142,01-22-2020.csv
2,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,NaN,NaN,30.0572,107.8740,01-22-2020.csv
3,Fujian,Mainland China,2020-01-22 17:00:00,1.0,NaN,NaN,26.0789,117.9874,01-22-2020.csv
4,Gansu,Mainland China,2020-01-22 17:00:00,NaN,NaN,NaN,36.0611,103.8343,01-22-2020.csv
5,Guangdong,Mainland China,2020-01-22 17:00:00,26.0,NaN,NaN,23.3417,113.4244,01-22-2020.csv
6,Guangxi,Mainland China,2020-01-22 17:00:00,2.0,NaN,NaN,23.8298,108.7881,01-22-2020.csv
7,Guizhou,Mainland China,2020-01-22 17:00:00,1.0,NaN,NaN,26.8154,106.8748,01-22-2020.csv
8,Hainan,Mainland China,2020-01-22 17:00:00,4.0,NaN,NaN,19.1959,109.7453,01-22-2020.csv
9,Hebei,Mainland China,2020-01-22 17:00:00,1.0,NaN,NaN,38.0428,114.5149,01-22-2020.csv
